# Problem 14.07 Liquid Nitrogen Production Via Volume Expansion of the Compressed Gas

Nitrogen at -104 °C and 250 bar flows through a valve to a pressure of 1 bar. What fraction of the stream becomes liquid?

## Solution

This is straightforward; energy is conserved and outlet presure is specified, making this a PH flash. This problem is also an important application that can show the results of different equations of state and how important good thermodynamics are. 

We can compare many different EOSs with Thermo easily.

In [1]:
from thermo import *
from thermo.interaction_parameters import SPDB
fluid = 'nitrogen'
constants, correlations = ChemicalConstantsPackage.from_IDs([fluid])

T1 = -104 + 273.15
P1 = 240*1e5
zs = [1]
P2 = 1e5

In [2]:
flasher_objects = []
flasher_names = []

gas = CoolPropGas('HEOS', fluid, T=T1, P=P1, zs=zs)
liquid = CoolPropLiquid('HEOS', fluid, T=T1, P=P1, zs=zs)
high_precision = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])
flasher_objects.append(high_precision)
flasher_names.append('High-Precision')

# Add the Peng-Robinson Pina-Martinez parameters EOS
Ls = SPDB.get_parameter_vector(name='PRTwu_PinaMartinez', CASs=constants.CASs, parameter='TwuPRL')
Ms = SPDB.get_parameter_vector(name='PRTwu_PinaMartinez', CASs=constants.CASs, parameter='TwuPRM')
Ns = SPDB.get_parameter_vector(name='PRTwu_PinaMartinez', CASs=constants.CASs, parameter='TwuPRN')
cs = SPDB.get_parameter_vector(name='PRTwu_PinaMartinez', CASs=constants.CASs, parameter='TwuPRc')
alpha_coeffs = [(Ls[i], Ms[i], Ns[i]) for i in range(constants.N)]
eos_kwargs = {'Pcs': constants.Pcs, 'Tcs': constants.Tcs, 'omegas': constants.omegas,
'cs': cs, 'alpha_coeffs':alpha_coeffs}

gas = CEOSGas(PRMIXTranslatedConsistent, eos_kwargs=eos_kwargs, HeatCapacityGases=correlations.HeatCapacityGases)
liquid = CEOSLiquid(PRMIXTranslatedConsistent, eos_kwargs=eos_kwargs, HeatCapacityGases=correlations.HeatCapacityGases)
eos_obj = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])
flasher_objects.append(eos_obj)
flasher_names.append('PR-Pina-Martinez')

# Add the SRK Pina-Martinez parameters EOS
Ls = SPDB.get_parameter_vector(name='SRKTwu_PinaMartinez', CASs=constants.CASs, parameter='TwuSRKL')
Ms = SPDB.get_parameter_vector(name='SRKTwu_PinaMartinez', CASs=constants.CASs, parameter='TwuSRKM')
Ns = SPDB.get_parameter_vector(name='SRKTwu_PinaMartinez', CASs=constants.CASs, parameter='TwuSRKN')
cs = SPDB.get_parameter_vector(name='SRKTwu_PinaMartinez', CASs=constants.CASs, parameter='TwuSRKc')
alpha_coeffs = [(Ls[i], Ms[i], Ns[i]) for i in range(constants.N)]
eos_kwargs = {'Pcs': constants.Pcs, 'Tcs': constants.Tcs, 'omegas': constants.omegas,
'cs': cs, 'alpha_coeffs':alpha_coeffs}

gas = CEOSGas(SRKMIXTranslatedConsistent, eos_kwargs=eos_kwargs, HeatCapacityGases=correlations.HeatCapacityGases)
liquid = CEOSLiquid(SRKMIXTranslatedConsistent, eos_kwargs=eos_kwargs, HeatCapacityGases=correlations.HeatCapacityGases)
eos_obj = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])
flasher_objects.append(eos_obj)
flasher_names.append('SRK-Pina-Martinez')

# Add a bunch of EOSs that don't require any parameters
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas)

cubic_EOSs = [('PR', PRMIX), ('SRK', SRKMIX), 
              ('VDW', VDWMIX),
             ('PRSV', PRSVMIX), ('PRSV2', PRSV2MIX),
             ('TWUPR', TWUPRMIX), ('TWUSRK', TWUSRKMIX),
             ('PRTranslatedConsistent', PRMIXTranslatedConsistent),
             ('SRKTranslatedConsistent', SRKMIXTranslatedConsistent)]
for eos_name, eos_obj in cubic_EOSs:
    liquid = CEOSLiquid(eos_obj, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
    gas = CEOSGas(eos_obj, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
    eos_obj = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])

    flasher_objects.append(eos_obj)
    flasher_names.append(eos_name)
    


In [3]:
for obj, obj_name in zip(flasher_objects, flasher_names):
    state_1 = obj.flash(T=T1, P=P1, zs=zs)
    state_2 = obj.flash(P=P2, H=state_1.H(), zs=zs)
    print(f'The {obj_name} EOS predicted liquid molar fraction is {state_2.LF:.8f}.')


The High-Precision EOS predicted liquid molar fraction is 0.03887228.
The PR-Pina-Martinez EOS predicted liquid molar fraction is 0.05514570.
The SRK-Pina-Martinez EOS predicted liquid molar fraction is 0.06744189.
The PR EOS predicted liquid molar fraction is 0.05852484.
The SRK EOS predicted liquid molar fraction is 0.04234207.
The VDW EOS predicted liquid molar fraction is 0.00000000.
The PRSV EOS predicted liquid molar fraction is 0.05903916.
The PRSV2 EOS predicted liquid molar fraction is 0.05903916.
The TWUPR EOS predicted liquid molar fraction is 0.05437623.
The TWUSRK EOS predicted liquid molar fraction is 0.04623410.
The PRTranslatedConsistent EOS predicted liquid molar fraction is 0.05738467.
The SRKTranslatedConsistent EOS predicted liquid molar fraction is 0.06963456.


As can be see, the equation of state used changes the results drastically. Even the best of the cubic equations of state given results 30-50% off from the high-precision equation of state. This problem was admittedly constructed to show off the importance of using higher precision models, but the point applies elsewhere also.